In [0]:

from pyspark.sql.window import Window
from pyspark.sql.functions import lead, col, broadcast

class Transformer:
    def __init__(self):
        pass

    def transform(self, inputDF):
        pass

class FirstTransformer(Transformer):

    def transform(self, inputDF):
        """
        Customers who have bought Airpods after buying the iPhone
        """
        transactionInputDF = inputDF.get("transactionInputDF")
        print("transactionInputDF in transform")
        transactionInputDF.show()
        windowSpec = Window.partitionBy("customer_id").orderBy("transaction_date")
        transformedDF = transactionInputDF.withColumn(
            "next_product_name", lead("product_name").over("windowSpec")
        )


        print("Airpods after buying iPhone")
        transformedDF.orderBy("customer_id","transaction_date","product_name").show()

        filteredDF = transformedDF.filter(
            (col("product_name") == "iPhone") & (col("next_product_name") == "Airpods")
            )
        
        filteredDF.orderBy("customer_id","transaction_date","product_name").show()

        customerInputDF = inputDF.get("customerInputDF")

        customerInputDF.show()

        joinDF = customerInputDF.join(
            broadcast(filteredDF),
            "customer_id"
        )

        print("Joined DF")
        joinDF.show()

        return joinDF.select(
            "customer_id",
            "customer_name",
            "location"
        )